# Issues

In [ ]:
/* crear tabla log
1)	
		id_log
		nombreLibro_anterior
		nombreLibro_actual
		id_libro
		fechaModificacion
			

-- guardar los datos, cuando se esten actualizando

	
2)	crear Sp guardar libro 
	Validar Nombre del libro y autor, no se deben repetir


*/

# Register updated books

Deprecated version

In [ ]:
CREATE PROCEDURE SP_ActualizarLibros
    @Id_libro int, 
    @Nombre_libro nvarchar(50), 
    @Autor nvarchar(50), 
    @Fecha_lanzamiento  datetime,
    @Usuario nvarchar(30)
AS      
BEGIN
    UPDATE "Books"
    SET 
         Nombre_libro = @Nombre_libro, 
         Autor = @Autor, 
         Fecha_lanzamiento = @Fecha_lanzamiento,
         Campos_auditoria = @Usuario  
    WHERE Id_libro = @Id_libro;
END;

Setup

In [ ]:
create table Logs (
    id_log INT IDENTITY PRIMARY KEY,
    nombreLibro_anterior varchar(60) NOT NULL,
    nombreLibro_actual varchar(60) NOT NULL,
    id_libro int NOT NULL,
    fechaModificacion datetime default getdate()
)

In [ ]:
create procedure SP_ActualizarLibros
    @Id_libro int, 
    @Nombre_libro varchar(50), 
    @Autor varchar(50), 
    @Fecha_lanzamiento  datetime,
    @Usuario varchar(30)
AS      
BEGIN
    begin try

        -- check if Nombre_libro was updated
        declare @nombre_anterior varchar(50);
        select @nombre_anterior=Nombre_libro from Books where Id_libro = @Id_libro;

        if @nombre_anterior = @Nombre_libro
            -- if not, just execute the update
            UPDATE "Books"
            SET 
                Nombre_libro = @Nombre_libro, 
                Autor = @Autor, 
                Fecha_lanzamiento = @Fecha_lanzamiento,
                Campos_auditoria = @Usuario  
                WHERE Id_libro = @Id_libro;

        else 
            -- if was, insert old name into Logs table
            INSERT INTO Logs (nombreLibro_anterior, nombreLibro_actual, id_libro)
            VALUES (@nombre_anterior, @Nombre_libro, @Id_libro)


            UPDATE "Books"
            SET 
                Nombre_libro = @Nombre_libro, 
                Autor = @Autor, 
                Fecha_lanzamiento = @Fecha_lanzamiento,
                Campos_auditoria = @Usuario  
            WHERE Id_libro = @Id_libro;

    end try

    begin catch
        
        select 'Something went wrong with the update' + ERROR_MESSAGE();

    end catch

END;

Check

In [ ]:
exec SP_ActualizarLibros
    @Id_libro=4,
    @Nombre_libro='The fourth one', 
    @Autor='DropDead', 
    @Fecha_lanzamiento='2023-02-12',
    @Usuario='dbAdmin'

In [41]:
select top(1) * from Books order by Id_libro desc
select * from Logs

(1 row affected)

(1 row affected)

Total execution time: 00:00:00.029

Id_libro,Nombre_libro,Autor,Fecha_lanzamiento,Campos_auditoria
4,The fourth one,DropDead,2023-12-02 00:00:00.0000000,dbAdmin


id_log,nombreLibro_anterior,nombreLibro_actual,id_libro,fechaModificacion
1,The fifth one,The fourth one,4,2023-12-21 09:04:33.750


How many title updates are per book?

In [ ]:
select b.Id_libro, count(l.nombreLibro_anterior) as title_changes 
from Books b
inner join Logs l
on l.id_libro = b.Id_libro
group by b.Id_libro
order by title_changes desc

# Avoid duplicates

Setup

In [ ]:
create procedure SP_CrearLibros
    @Nombre_libro varchar(50), 
    @Autor varchar(50), 
    @Fecha_lanzamiento  datetime,
    @Usuario varchar(30)

as begin

    begin try
        -- book name validation
        if exists(select Nombre_libro from Books where Nombre_libro = @Nombre_libro)
            RAISERROR('You cant, book name already exists', 16, 1);

        -- author name validation
        if exists(select Autor from Books where Autor = @Autor)
            RAISERROR('You cant, author name already exists', 16, 1);

        insert into Books (Nombre_libro, Autor, Fecha_lanzamiento, Campos_auditoria)
            values (@Nombre_libro, @Autor, @Fecha_lanzamiento, @Usuario)

        -- operation output message
        select 'correctly inserted' Mensaje, 1 Bandera

    end try

    begin catch

        select '' + ERROR_MESSAGE() Mensaje, 0 Bandera,
        ERROR_LINE() at_line,
        ERROR_PROCEDURE() procedure_name,
        ERROR_NUMBER() error_number

    end catch
end

Check

In [ ]:
exec SP_CrearLibros
@Nombre_libro = 'The fifth one',
@Autor = 'DropDead',
@Fecha_lanzamiento = '2002-02-09',
@Usuario = 'DB'

In [39]:
select * from Books

(4 rows affected)

Total execution time: 00:00:00.006

Id_libro,Nombre_libro,Autor,Fecha_lanzamiento,Campos_auditoria
1,The first one,Cohelo,2023-12-21 08:57:00.0000000,Paco
2,The second one,Re,2023-12-21 08:58:00.0000000,Alfonso
3,The third one,Kojima,2023-12-21 08:58:00.0000000,Eduardo
4,The fourth one,DropDead,2023-12-02 00:00:00.0000000,dbAdmin
